In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
subm_csv = pd.read_csv('../input/hpapublic-csv/submission.csv')

In [ ]:
list_subm = subm_csv.ID.to_list()

In [ ]:
!pip install /kaggle/input/kerasapplications -q
!pip install /kaggle/input/efficientnet-keras-source-code/ -q --no-deps
!pip install -q "/kaggle/input/pycocotools/pycocotools-2.0-cp37-cp37m-linux_x86_64.whl"
!pip install -q "/kaggle/input/hpapytorchzoozip/pytorch_zoo-master"
!pip install -q "/kaggle/input/hpacellsegmentatormaster/HPA-Cell-Segmentation-master"

#!wget "https://www.kaggleusercontent.com/kf/60257229/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..CiJyNbJXntOyTTiedEu0RA.9ehsk6hfDsNYbo2hw344h9FJN_gNQ_7JbmG5addEB5yMS2GEVsiYEbCYtTZrl_GWl2rm_BlEfgGn-uvzr1m1MsAgen97qAVX-FG6bCguUWThX4Sj1YbOtZYUO_bdvn63ce9n18_BC5CB2BfcmjCXIEamsaM-LF0PlOmlJ42WSrnyX7A3sUlBAV8AXgaeku0F17F3qEGClYYro9kQPAgT4Wg2zQVegDNtpE_52kj0mOVagAoWPw5N8sD2ueY4HrY4uTgNvQwHKQq60ce3Wu0XVODROcpZZSDIpfarofwUZQhSe4GmnoxHJA4kBN_kG1_0YdnsvOKCVmLoVC7BmChqPr-9llQhyT6pBVRIhCBXanH2dYHy-bLxZPXXmH-bYgVuxNTt3wKR-LcVScafukQfwg_tHBOAxb5U7a7ZuV7zfDY5HjwMrhjdd0JkerXrBCvOhsC5NQ4H9eIUB0csEZhQwEQmGk6u9w0NvDS-7gdz71b-qsI7Atk4TNTa9yEoJbgXZH6kRRHlb9kpjA3XCPcnV5j0Mwjli-qdqJj_uXwd66CLpy9wvnRaniqm8bOlf7wVfGCXZqdEjFp1WlYcWuoKKdsMZv-6-SVXA52_suOaMq1pf7HodgDobeohQBGZQuzLv_xRZnOUZPlwAE0OZa_vB9W7a48mwPWnS6Lh0llFaHk.3bLf-RBpLBwqm6-8VMCrwA/model_green.h5"

In [ ]:
import hpacellseg.cellsegmentator as cellsegmentator
from hpacellseg.utils import label_cell, label_nuclei
import glob
import tensorflow_addons as tfa

In [ ]:
ONLY_PUBLIC = False

In [ ]:
import base64
from pycocotools import mask as mutils
from pycocotools import _mask as coco_mask
import typing as t
import zlib
def encode_binary_mask(mask: np.ndarray) -> t.Text:
  """Converts a binary mask into OID challenge encoding ascii text."""

  # check input mask --
  if mask.dtype != np.bool:
    raise ValueError(
        "encode_binary_mask expects a binary mask, received dtype == %s" %
        mask.dtype)

  mask = np.squeeze(mask)
  if len(mask.shape) != 2:
    raise ValueError(
        "encode_binary_mask expects a 2d mask, received shape == %s" %
        mask.shape)

  # convert input mask to expected COCO API input --
  mask_to_encode = mask.reshape(mask.shape[0], mask.shape[1], 1)
  mask_to_encode = mask_to_encode.astype(np.uint8)
  mask_to_encode = np.asfortranarray(mask_to_encode)

  # RLE encode mask --
  encoded_mask = coco_mask.encode(mask_to_encode)[0]["counts"]

  # compress and base64 encoding --
  binary_str = zlib.compress(encoded_mask, zlib.Z_BEST_COMPRESSION)
  base64_str = base64.b64encode(binary_str)
  return base64_str.decode()

In [ ]:
import matplotlib.pyplot as plt
import cv2

In [ ]:
def decreasing_non_label(pred):
  '''
  expect shape(dls.c,)
  '''
  #pred = pred.numpy()
  avg = torch.mean(pred)
  
  pred[pred<avg] -= 4
  
  return pred


In [ ]:
import tensorflow as tf
import efficientnet.tfkeras as efn
from tqdm import tqdm

In [ ]:
def rle_encode_less_memory(img):
    pixels = img.T.flatten()
    pixels[0] = 0
    pixels[-1] = 0
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 2
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)
def enc2mask(encs, shape):
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for m,enc in enumerate(encs):
        if isinstance(enc,np.float) and np.isnan(enc): continue
        s = enc.split()
        for i in range(len(s)//2):
            start = int(s[2*i]) - 1
            length = int(s[2*i+1])
            img[start:start+length] = 1 + m
    return img.reshape(shape).T

In [ ]:
import sys

In [ ]:
from tensorflow import keras

In [ ]:
def macro_f1(y, y_hat, thresh=0.5):
    """Compute the macro F1-score on a batch of observations (average F1 across labels)
    
    Args:
        y (int32 Tensor): labels array of shape (BATCH_SIZE, N_LABELS)
        y_hat (float32 Tensor): probability matrix from forward propagation of shape (BATCH_SIZE, N_LABELS)
        thresh: probability value above which we predict positive
        
    Returns:
        macro_f1 (scalar Tensor): value of macro F1 for the batch
    """
    y_pred = tf.cast(tf.greater(y_hat, thresh), tf.float32)
    tp = tf.cast(tf.math.count_nonzero(y_pred * y, axis=0), tf.float32)
    fp = tf.cast(tf.math.count_nonzero(y_pred * (1 - y), axis=0), tf.float32)
    fn = tf.cast(tf.math.count_nonzero((1 - y_pred) * y, axis=0), tf.float32)
    f1 = 2*tp / (2*tp + fn + fp + 1e-16)
    macro_f1 = tf.reduce_mean(f1)
    return macro_f1

In [ ]:
import os.path
import urllib
import zipfile

import numpy as np
import scipy.ndimage as ndi
from skimage import filters, measure, segmentation
from skimage.morphology import (binary_erosion, closing, disk,
                                remove_small_holes, remove_small_objects)
import torch
HIGH_THRESHOLD = 0.4
LOW_THRESHOLD = HIGH_THRESHOLD - 0.25


def download_with_url(url_string, file_path, unzip=False):
    """Download file with a link."""
    with urllib.request.urlopen(url_string) as response, open(
        file_path, "wb"
    ) as out_file:
        data = response.read()  # a `bytes` object
        out_file.write(data)

    if unzip:
        with zipfile.ZipFile(file_path, "r") as zip_ref:
            zip_ref.extractall(os.path.dirname(file_path))


def __fill_holes(image):
    """Fill_holes for labelled image, with a unique number."""
    boundaries = segmentation.find_boundaries(image)
    image = np.multiply(image, np.invert(boundaries))
    image = ndi.binary_fill_holes(image > 0)
    image = ndi.label(image)[0]
    return image





def label_cell2(nuclei_pred, cell_pred):
    """Label the cells and the nuclei.
    Keyword arguments:
    nuclei_pred -- a 3D numpy array of a prediction from a nuclei image.
    cell_pred -- a 3D numpy array of a prediction from a cell image.
    Returns:
    A tuple containing:
    nuclei-label -- A nuclei mask data array.
    cell-label  -- A cell mask data array.
    0's in the data arrays indicate background while a continous
    strech of a specific number indicates the area for a specific
    cell.
    The same value in cell mask and nuclei mask refers to the identical cell.
    NOTE: The nuclei labeling from this function will be sligthly
    different from the values in :func:`label_nuclei` as this version
    will use information from the cell-predictions to make better
    estimates.
    """
    def __wsh(
        mask_img,
        threshold,
        border_img,
        seeds,
        threshold_adjustment=0.35,
        small_object_size_cutoff=10,
    ):
        img_copy = np.copy(mask_img)
        m = seeds * border_img  # * dt
        img_copy[m <= threshold + threshold_adjustment] = 0
        img_copy[m > threshold + threshold_adjustment] = 1
        img_copy = img_copy.astype(np.bool)
        img_copy = remove_small_objects(img_copy, small_object_size_cutoff).astype(
            np.uint8
        )

        mask_img[mask_img <= threshold] = 0
        mask_img[mask_img > threshold] = 1
        mask_img = mask_img.astype(np.bool)
        mask_img = remove_small_holes(mask_img, 63)
        mask_img = remove_small_objects(mask_img, 1).astype(np.uint8)
        markers = ndi.label(img_copy, output=np.uint32)[0]
        labeled_array = segmentation.watershed(
            mask_img, markers, mask=mask_img, watershed_line=True
        )
        return labeled_array

    nuclei_label = __wsh(
        nuclei_pred[..., 2] / 255.0,
        0.4,
        1 - (nuclei_pred[..., 1] + cell_pred[..., 1]) / 255.0 > 0.05,
        nuclei_pred[..., 2] / 255,
        threshold_adjustment=-0.25,
        small_object_size_cutoff=32,
    )

    # for hpa_image, to remove the small pseduo nuclei
    nuclei_label = remove_small_objects(nuclei_label, 157)
    nuclei_label = measure.label(nuclei_label)
    # this is to remove the cell borders' signal from cell mask.
    # could use np.logical_and with some revision, to replace this func.
    # Tuned for segmentation hpa images
    threshold_value = max(0.22, filters.threshold_otsu(cell_pred[..., 2] / 255) * 0.5)
    # exclude the green area first
    cell_region = np.multiply(
        cell_pred[..., 2] / 255 > threshold_value,
        np.invert(np.asarray(cell_pred[..., 1] / 255 > 0.05, dtype=np.int8)),
    )
    sk = np.asarray(cell_region, dtype=np.int8)
    distance = np.clip(cell_pred[..., 2], 255 * threshold_value, cell_pred[..., 2])
    cell_label = segmentation.watershed(-distance, nuclei_label, mask=sk)
    cell_label = remove_small_objects(cell_label, 344).astype(np.uint8)
    selem = disk(2)
    cell_label = closing(cell_label, selem)
    cell_label = __fill_holes(cell_label)
    # this part is to use green channel, and extend cell label to green channel
    # benefit is to exclude cells clear on border but without nucleus
    sk = np.asarray(
        np.add(
            np.asarray(cell_label > 0, dtype=np.int8),
            np.asarray(cell_pred[..., 1] / 255 > 0.05, dtype=np.int8),
        )
        > 0,
        dtype=np.int8,
    )
    cell_label = segmentation.watershed(-distance, cell_label, mask=sk)
    cell_label = __fill_holes(cell_label)
    cell_label = np.asarray(cell_label > 0, dtype=np.uint8)
    cell_label = measure.label(cell_label)
    cell_label = remove_small_objects(cell_label, 344) # was 344
    cell_label = measure.label(cell_label)
    cell_label = np.asarray(cell_label, dtype=np.uint16)
    #nuclei_label = np.multiply(cell_label > 0, nuclei_label) > 0
    #nuclei_label = measure.label(nuclei_label)
    #nuclei_label = remove_small_objects(nuclei_label, 157)
    #nuclei_label = np.multiply(cell_label, nuclei_label > 0)

    return cell_label#nuclei_label, cell_label

In [ ]:
!mkdir cell_tile
cell_dir = 'cell_tile'

In [ ]:
NUC_MODEL = "../input/hpacellsegmentatormodelweights/dpn_unet_nuclei_v1.pth"
CELL_MODEL = "../input/hpacellsegmentatormodelweights/dpn_unet_cell_3ch_v1.pth"
segmentator = cellsegmentator.CellSegmentator(
        NUC_MODEL,
        CELL_MODEL,
        scale_factor=0.25,
        device="cuda",
        padding=True,
        multi_channel_model=True,
    )
i_df = pd.read_csv("../input/hpa-single-cell-image-classification/sample_submission.csv")
if ONLY_PUBLIC==True:
    i_df = pd.read_csv("../input/hpa-sample-submission-with-extra-metadata/updated_sample_submission.csv")
bs = 40
bs_now=0
image_list=[]
mt = []
er = []
nu = []
ID_list=[]
for i in tqdm(i_df.index):
    ID = i_df.loc[i,'ID']
    #if len(i_df)==559:
    #    continue
    if len(glob.glob('../input/hpscell-tile-in-image/cell_tile/'+ID+'.npz'))>0:
        continue
    W= int(i_df[i_df['ID']==ID]['ImageWidth'].values[0])
    H= int(i_df[i_df['ID']==ID]['ImageHeight'].values[0])
    ID_list.append(ID)
    name = '../input/hpa-single-cell-image-classification/test' + '/' +ID+ '_red.png'
    mt.append(name)
    er.append(name.replace('red', 'yellow'))
    nu.append(name.replace('red', 'blue'))
    images = [mt, er, nu]
    bs_now+=1
    if bs_now>=bs:
        
        # For nuclei
        nuc_segmentations = segmentator.pred_nuclei(images[2])

        # For full cells
        cell_segmentations = segmentator.pred_cells(images)

        
        
        
        cell_mask=[]
        for j, pred in enumerate(cell_segmentations):
            size = cell_segmentations[j].shape[0]
            cell_mask_ = label_cell2(cv2.resize(nuc_segmentations[j],(512,512)), cv2.resize(cell_segmentations[j],(512,512)))
            cell_mask_ = cv2.resize(cell_mask_,(size,size),interpolation=cv2.INTER_NEAREST)
            cell_mask.append(cell_mask_)
        for id_,mask in zip(ID_list,cell_mask):
            np.savez_compressed(f'{cell_dir}/{id_}', mask)
        
        # clear
        
        image_list=[]
        mt = []
        er = []
        nu = []
        ID_list=[]
        bs_now=0
if bs_now>0:
        nuc_segmentations = segmentator.pred_nuclei(images[2])

        # For full cells
        cell_segmentations = segmentator.pred_cells(images)

        
        
        cell_mask=[]
        for j, pred in enumerate(cell_segmentations):
            size = cell_segmentations[j].shape[0]
            cell_mask_ = label_cell2(cv2.resize(nuc_segmentations[j],(512,512)), cv2.resize(cell_segmentations[j],(512,512)))
            cell_mask_ = cv2.resize(cell_mask_,(size,size),interpolation=cv2.INTER_NEAREST)
            cell_mask.append(cell_mask_)
        for id_,mask in zip(ID_list,cell_mask):
            np.savez_compressed(f'{cell_dir}/{id_}', mask)
        
        # clear
        
        image_list=[]
        mt = []
        er = []
        nu = []
        ID_list=[]
        bs_now=0

In [ ]:
import gc
del CELL_MODEL
del NUC_MODEL
del cellsegmentator
#del NUC
gc.collect()
torch.cuda.empty_cache()

In [ ]:
'''
try:
            tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
            tf.config.experimental_connect_to_cluster(tpu)
            tf.tpu.experimental.initialize_tpu_system(tpu)
            strategy = tf.distribute.experimental.TPUStrategy(tpu)
            print("Running on TPU:", tpu.master())
except ValueError:
            strategy = tf.distribute.get_strategy()
            print(f"Running on {strategy.num_replicas_in_sync} replicas")
'''


'''
model = tf.keras.models.load_model(
            '../input/hpa-classification-efnb7-train/model_green.h5',compile = False
        )
'''

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
        try:
            # Currently, memory growth needs to be the same across GPUs
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "... Physical GPUs,", len(logical_gpus), "Logical GPUs ...\n")
        except RuntimeError as e:
            # Memory growth must be set before GPUs have been initialized
            print(e)


model = tf.keras.models.load_model(
            '../input/download-model/model_RGB_0.0502.h5'
        )
'''
model_4 = tf.keras.models.load_model(
            '../input/hpa-training-tpu-pseude-label-t/model_RGB_0.0635.h5'
        )
model_2 = tf.keras.models.load_model(
            '../input/atlasmodel/model_RGB.h5'
        )
    
model_3 = tf.keras.models.load_model('../input/hpa-training-tpu/model_RGB_0.2217.h5')
#model_4 = tf.keras.models.load_model('../input/atlasmodel/model_RGB_0.1139.h5')
model_g = tf.keras.models.load_model('../input/fork-of-hpa-training-tpu-2/model_RGB_0.2440.h5')
'''
'''
model_whole = tf.keras.models.load_model(
            '../input/hpa-classification-efnb7-train-tfrec/model_green.h5'
        )
'''
model_whole_2 = tf.keras.models.load_model(
            '../input/atlasmodel/model_green.h5'
        )
'''
model_whole_3 = tf.keras.models.load_model(
            '../input/download-model3/model_green.h5'
        )
model_whole_4 = tf.keras.models.load_model(
            '../input/hpa-classification-efnb7-train-noisy/model_green.h5'
        )
'''

In [ ]:
# for TF model
import time
NUC_MODEL = "../input/hpacellsegmentatormodelweights/dpn_unet_nuclei_v1.pth"
CELL_MODEL = "../input/hpacellsegmentatormodelweights/dpn_unet_cell_3ch_v1.pth"

i_df = pd.read_csv("../input/hpa-single-cell-image-classification/sample_submission.csv")
if len(i_df)==559:
 with open('submission.csv', 'w') as outf:
    print('ID,ImageWidth,ImageHeight,PredictionString', file=outf)
 print('ready to submit')
 
else:
 if ONLY_PUBLIC==True:
    i_df = pd.read_csv("../input/hpa-sample-submission-with-extra-metadata/updated_sample_submission.csv")
 with open('submission.csv', 'w') as outf:
  print('ID,ImageWidth,ImageHeight,PredictionString', file=outf)
  for ind in tqdm(i_df.index):
    
    ID = i_df.iloc[ind]["ID"]
    W = i_df.iloc[ind]["ImageWidth"]
    H = i_df.iloc[ind]["ImageHeight"]
    
    if ID in list_subm:
        ps = subm_csv[subm_csv['ID']==ID]['PredictionString'].values[0]
        #print(subm_csv[subm_csv['ID']==ID]['PredictionString'], file=outf)
        print(f'{ID},{W},{H},{ps}', file=outf)
        continue
    
    green = plt.imread('../input/hpa-single-cell-image-classification/test/'+ID+'_green.png')
    red = plt.imread('../input/hpa-single-cell-image-classification/test/'+ID+'_red.png')
    blue = plt.imread('../input/hpa-single-cell-image-classification/test/'+ID+'_blue.png')
    green = green[:,:,np.newaxis]
    
    blue = blue[:,:,np.newaxis]
    red = red[:,:,np.newaxis]
    all_ = np.concatenate((red,green,blue),axis=-1)
    del red,blue
    
    
    w = i_df.iloc[ind]["ImageWidth"]
    h = i_df.iloc[ind]["ImageHeight"]
    '''
    mt = glob.glob('../input/hpa-single-cell-image-classification/test' + '/' +ID+ '_red.png')
    er = [f.replace('red', 'yellow') for f in mt]
    nu = [f.replace('red', 'blue') for f in mt]
    images = [mt, er, nu]
    '''
    #ensemble the image-wise
    all_whole = np.concatenate((green,green,green),axis=-1)
    all_whole_720 = cv2.resize(all_whole,(720,720),
                              interpolation = cv2.INTER_AREA)
    all_whole_720 = np.expand_dims(all_whole_720,axis=0)
    all_whole_512 = cv2.resize(all_whole,(512,512),
                              interpolation = cv2.INTER_AREA)
    all_whole_512 = np.expand_dims(all_whole_512,axis=0)
    all_whole = cv2.resize(all_whole,(600,600),
                              interpolation = cv2.INTER_AREA)
    
    all_whole = np.expand_dims(all_whole,axis=0)
    #pred_whole = model_whole.predict(all_whole)
    pred_whole2 = model_whole_2.predict(all_whole)
    #pred_whole3 = model_whole_3.predict(all_whole_720)
    #pred_whole4 = model_whole_4.predict(all_whole)
    pred_whole = (pred_whole2)/1.
    
    del all_whole,all_whole_720,all_whole_512
    
    # For nuclei
    #nuc_segmentations = segmentator.pred_nuclei(images[2])

    # For full cells
    #cell_segmentations = segmentator.pred_cells(images)

    # post-processing
    
    
    #nuclei_mask, cell_mask = label_cell(nuc_segmentations[0], cell_segmentations[0])
    #FOVname = os.path.basename(mt[i]).replace('red','predictedmask')
    #imageio.imwrite(os.path.join(save_dir,FOVname), cell_mask) 
    file_ =glob.glob('../input/hpscell-tile-in-image/cell_tile/'+ID+'.npz')
    if len(file_)>0:
        cell_mask = np.load(file_[0])['arr_0']
    else:
        cell_mask = np.load(f'{cell_dir}/{ID}.npz')['arr_0']
    pred_strs = []
    
    for i in range(1,cell_mask.max()+1):
          x,y=(cell_mask==i).nonzero()
          cell_mask_s = cell_mask==i
          cell_mask_s_ = cell_mask_s[:,:,np.newaxis]
          i_c = cell_mask_s_ * all_
          
            
            
            
          i_c_720 = cv2.resize(i_c,(720,720),
                              interpolation = cv2.INTER_AREA)
          i_c_720 = np.stack([i_c_720[:,:,1],i_c_720[:,:,1],i_c_720[:,:,1]],axis=2)
          i_c_720 = np.expand_dims(i_c_720,axis=0)  
          
          i_c = cv2.resize(i_c,(600,600),
                              interpolation = cv2.INTER_AREA)
          i_c = np.stack([i_c[:,:,1],i_c[:,:,1],i_c[:,:,1]],axis=2)
          i_c = np.expand_dims(i_c,axis=0)
          
            
            
        
          pred_ic = model_whole_2.predict(i_c)
          #pred_ic2 = model_whole.predict(i_c)
          #pred_ic3 = model_whole_3.predict(i_c_720)
          #print(pred_ic)
        
          del i_c,i_c_720
          
          lenx=x.max()-x.min()+1
          leny=y.max()-y.min()+1
          xx=x.min()
          yy=y.min()

          interest=cell_mask[xx:xx+lenx,yy:yy+leny]
          all_g=all_[xx:xx+lenx,yy:yy+leny]
          if lenx>leny:
              pad=lenx-leny
              interest=np.pad(interest,((0,0),(pad//2,pad-pad//2)), 'constant', constant_values=(0))
              all_g=np.pad(all_g,((0,0),(pad//2,pad-pad//2),(0,0)), 'constant', constant_values=(0))
          elif leny>lenx:
              pad=leny-lenx
              interest=np.pad(interest,((pad//2,pad-pad//2),(0,0)), 'constant', constant_values=(0))
              all_g=np.pad(all_g,((pad//2,pad-pad//2),(0,0),(0,0)), 'constant', constant_values=(0))
          interest = interest[:,:,np.newaxis]
          interest=(interest==i)

          all_g=interest*all_g

          
          del interest
          

          




          all_g = cv2.resize(all_g,(256,256),
                              interpolation = cv2.INTER_AREA)
          #all_g = np.transpose(all_g,(2,0,1))
          all_green = np.stack([all_g[:,:,1],all_g[:,:,1],all_g[:,:,1]],axis=2)
          all_g = np.expand_dims(all_g,axis=0)
          all_green = np.expand_dims(all_green,axis=0)
          #all_g=(torch.from_numpy(all_g)).to(device)
          #model.eval()
          pred_c  = 0

          
          pred = model.predict(all_g)
          #pred2 = model_2.predict(all_g)
          #pred3 = model_3.predict(all_g)
          #pred4 = model_4.predict(all_g)
          #pred4 = model_4.predict(all_g)
          #predg = model_g.predict(all_green)
          pred = (pred+pred_ic)/2.
          # Only get the argmax one
          pred = pred*0.6 + pred_whole*0.4
          pred_c = pred[0]
            
          del all_g,all_green
          
          #pred_c = decreasing_non_label(pred_c)
            
          rle = encode_binary_mask(cell_mask_s)
          
            
          #set non-green to class 18
          '''
          green_channel = all_g[:,:,1]
          green_channel[green_channel<0.1]=0.
          if np.sum(green_channel)<0.5:
                pred_c = [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.0]
          '''
          
          
          for class_id in range(19):
            #cnf = torch.sigmoid(pred_c[class_id])
            cnf = pred_c[class_id]
            pred_strs.append(f'{class_id} {cnf} {rle}')
            
    print(f'{ID},{w},{h},{" ".join(pred_strs)}', file=outf)
    gc.collect()
    
'''
local_file = 'submission.csv'

df_submit = pd.read_csv('../input/hpa-single-cell-image-classification/sample_submission.csv', index_col='ID')
df_local  = pd.read_csv(local_file, index_col='ID')
df_local=df_local[['ImageWidth','ImageHeight','PredictionString']]
df_submit.loc[df_local.index.values] = df_local.values  


df_submit.to_csv('submission.csv')
'''
